#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 data and use them to predict the outcome of cases from the last 30 days

##Install the package from github using pip

In [1]:
#This library is only needed for the Cox Regression Model, which is not included in this tutorial
#! pip install lifelines

In [2]:
#pwd()

In [3]:
#! pip install ../

##Import the Boston311Model class

In [ ]:
! pip show boston311

: 

In [5]:
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree

2023-09-07 12:59:17.377934: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-07 12:59:17.874861: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-07 12:59:17.879069: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 12:59:20.992056: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


##Get latest file URLS and Current Date Ranges

In [6]:
logregmodel = Boston311LogReg()
logregmodel.load('logregproperties.json', 'logreg.h5')

In [7]:
from datetime import datetime, timedelta
now = datetime.now()
thirty_days = timedelta(days=30)
thirty_days_ago = now - thirty_days
today_datestring = now.strftime("%Y-%m-%d")
thirty_days_ago_datestring = thirty_days_ago.strftime("%Y-%m-%d")

print(today_datestring, thirty_days_ago_datestring)

2023-09-07 2023-08-08


In [8]:
logregmodel.predict_date_range = {'start': thirty_days_ago_datestring, 'end': today_datestring}

In [9]:
logregmodel_prediction = logregmodel.predict()

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:251: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0]
918/918 [==============================] - 4s 4ms/step


In [10]:
logregmodel_prediction.head()

,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,closure_reason,case_title,subject,reason,...,location_zipcode,latitude,longitude,geom_4326,source,survival_time,event,ward_number,survival_time_hours,event_prediction
26522,101005014325,2023-08-26 06:18:00,2023-08-29 04:30:00,2023-08-26 14:15:10,ONTIME,Closed,Case Closed. Closed date : Sat Aug 26 18:15:10...,Requests for Street Cleaning,Public Works Department,Street Cleaning,...,2111.0,42.350910,-71.057170,0101000020E61000009C044BAEA8C351C0CA74669AEA2C...,Citizens Connect App,0 days 07:57:10,1,3,7.952778,1.000000
27919,101005014359,2023-08-26 06:32:00,2023-08-29 04:30:00,2023-08-28 07:14:36,ONTIME,Closed,Case Closed. Closed date : 2023-08-28 11:14:36...,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,NaN,42.307148,-71.082349,0101000020E61000003939283345C551C0032BE6A05027...,Constituent Call,2 days 00:42:36,1,14,48.710000,0.998558
27970,101005014366,2023-08-26 06:37:08,2023-08-29 04:30:00,2023-08-28 03:43:16,ONTIME,Closed,Case Closed. Closed date : Mon Aug 28 07:43:16...,Requests for Street Cleaning,Public Works Department,Street Cleaning,...,2136.0,42.258130,-71.121401,0101000020E6100000513CEA07C5C751C097AF2D640A21...,Citizens Connect App,1 days 21:06:08,1,18,45.102222,0.999998
28344,101005014372,2023-08-26 06:38:46,2023-08-29 04:30:00,2023-08-26 09:32:56,ONTIME,Closed,Case Closed. Closed date : Sat Aug 26 13:32:56...,Requests for Street Cleaning,Public Works Department,Street Cleaning,...,2116.0,42.344750,-71.074471,0101000020E61000007D54F91FC4C451C08C8AC4C0202C...,Citizens Connect App,0 days 02:54:10,1,4,2.902778,0.999662
28365,101005014400,2023-08-26 06:49:00,NaN,NaT,ONTIME,Open,,Mattress Pickup,Public Works Department,Sanitation,...,2125.0,42.316170,-71.056941,0101000020E6100000A207EDE9A4C351C0C57C2D3E7828...,Constituent Call,NaT,0,13,NaN,1.000000


In [11]:
import pandas as pd

ml_model_name = 'logregmodel'
ml_model_type = 'logistic regression'
# set ml_model_id to 1 and then today_datestring as an integer, to make a string of digits
ml_model_id = '1' + today_datestring.replace('-','')

#create a new dataframe with new columns, ml_model_name, ml_model_type,ml_model_id
ml_model_df = pd.DataFrame({'ml_model_name': [ml_model_name], 
                                    'ml_model_type': [ml_model_type],
                                    'id': [ml_model_id],
                                    'ml_model_date': [today_datestring]})




In [12]:
#get new dataframe with just the event_prediction column from the logregmodel_prediction dataframe
logregmodel_prediction_event = logregmodel_prediction[['event_prediction','case_enquiry_id']].copy()
logregmodel_prediction_event['ml_model_id'] = ml_model_id
#add today's date to the dataframe
logregmodel_prediction_event['prediction_date'] = today_datestring
#rename the event_prediction column to prediction
logregmodel_prediction_event.rename(columns={'event_prediction': 'prediction'}, inplace=True)


In [13]:
#remove logregmodel_prediction event_prediction column
logregmodel_cases = logregmodel_prediction.drop('event_prediction', axis=1).copy()

In [14]:
#remove geom column in logregmodel_cases
logregmodel_cases = logregmodel_cases.drop('geom_4326', axis=1).copy()

In [15]:
logregmodel_cases.head()

,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,closure_reason,case_title,subject,reason,...,precinct,location_street_name,location_zipcode,latitude,longitude,source,survival_time,event,ward_number,survival_time_hours
26522,101005014325,2023-08-26 06:18:00,2023-08-29 04:30:00,2023-08-26 14:15:10,ONTIME,Closed,Case Closed. Closed date : Sat Aug 26 18:15:10...,Requests for Street Cleaning,Public Works Department,Street Cleaning,...,0308,116 South St,2111.0,42.350910,-71.057170,Citizens Connect App,0 days 07:57:10,1,3,7.952778
27919,101005014359,2023-08-26 06:32:00,2023-08-29 04:30:00,2023-08-28 07:14:36,ONTIME,Closed,Case Closed. Closed date : 2023-08-28 11:14:36...,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,1403,INTERSECTION Bilodeau Rd & Washington St,NaN,42.307148,-71.082349,Constituent Call,2 days 00:42:36,1,14,48.710000
27970,101005014366,2023-08-26 06:37:08,2023-08-29 04:30:00,2023-08-28 03:43:16,ONTIME,Closed,Case Closed. Closed date : Mon Aug 28 07:43:16...,Requests for Street Cleaning,Public Works Department,Street Cleaning,...,1813,28 Webster St,2136.0,42.258130,-71.121401,Citizens Connect App,1 days 21:06:08,1,18,45.102222
28344,101005014372,2023-08-26 06:38:46,2023-08-29 04:30:00,2023-08-26 09:32:56,ONTIME,Closed,Case Closed. Closed date : Sat Aug 26 13:32:56...,Requests for Street Cleaning,Public Works Department,Street Cleaning,...,0401,63 Dartmouth St,2116.0,42.344750,-71.074471,Citizens Connect App,0 days 02:54:10,1,4,2.902778
28365,101005014400,2023-08-26 06:49:00,NaN,NaT,ONTIME,Open,,Mattress Pickup,Public Works Department,Sanitation,...,1308,1015 Dorchester Ave,2125.0,42.316170,-71.056941,Constituent Call,NaT,0,13,NaN


##Save the prediction data

In [16]:
logregmodel_cases.to_csv('311.csv', index=False)


In [17]:

logregmodel_prediction_event.to_csv('311_predictions.csv', index=False)

In [18]:

ml_model_df.to_csv('311_ml_models.csv', index=False)